In [5]:
import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline

In [2]:
import spacy
df = pd.read_csv('../data/books.csv')
nlp = spacy.load('en_core_web_sm')
def clean_description(text):
    doc = nlp(text.lower())
    return [token.lemma_ for token in doc if not token.is_punct and not token.is_stop]
df['cleaned_tokens'] = df['description'].apply(clean_description)
df['cleaned_text'] = df['cleaned_tokens'].apply(lambda tokens: ' '.join(tokens))

In [3]:
X = df['cleaned_text']
y = df['favorite_genre']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

Now, instead of using fit() and fit_transform(), we'll create a pipeline, which is not messy and works will with new data.

In [6]:
text_clf = Pipeline([('tfidf',TfidfVectorizer()),('clf',LogisticRegression())])
# tfidf is for Tfidf vectorization, and clf is to apply logistic regression classifier

In [7]:
# Training the pipeline:
text_clf.fit(X_train,y_train)
y_pred = text_clf.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

     Fantasy       0.00      0.00      0.00       1.0
      Horror       0.00      0.00      0.00       0.0

    accuracy                           0.00       1.0
   macro avg       0.00      0.00      0.00       1.0
weighted avg       0.00      0.00      0.00       1.0



C:\Users\anuro\miniconda3\envs\neurotale\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\anuro\miniconda3\envs\neurotale\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\anuro\miniconda3\envs\neurotale\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [13]:
params = {
    'tfidf__max_df': [0.9, 1.0],
    'tfidf__min_df': [1, 2],
    'clf__C': [0.1, 1, 10]
}

grid = GridSearchCV(text_clf, param_grid=params, cv=2)
grid.fit(X_train, y_train)

print("Best parameters found:", grid.best_params_)
print("Best score:", grid.best_score_)

ValueError: n_splits=2 cannot be greater than the number of members in each class.

In [14]:
import joblib
joblib.dump(grid.best_estimator_,'../models/final_model.pkl')
model = joblib.load('../models/final_model.pkl')
model.predict(['haunted house and ghosts'])

AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'